# Asignaciones financieras


## Datos de proyecto en análisis


In [128]:
proyecto = "misanayoc"
mes = 5
anio = 2025

## Librerias necesarias


In [129]:
import pandas as pd
import os
import pprint
import pickle
import json
import xlsxwriter
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import re
from typing import Dict, List, Any
import itertools
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from google.oauth2 import service_account
from google.cloud import firestore
import excel2img
from dotenv import load_dotenv, find_dotenv
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import win32com.client as win32

## Funciones utiles


### Calculadora de costo total


In [130]:
from typing import TypedDict


class CostoIngenieriaResult(TypedDict):
    costo_directo: float
    gastos_generales: float
    utilidad: float
    subtotal: float
    igv: float
    total: float


def calculadora_costo_total(costo_directo: float, gastos_generales: float) -> CostoIngenieriaResult:
    """
    Calcula el costo total de un proyecto de ingeniería civil en soles peruanos.

    Args:
        costo_directo (float): Costo directo del proyecto

    Returns:
        CostoIngenieriaResult: Diccionario con todos los valores calculados
    """
    # Calcular gastos generales (10% del costo directo)
    gastos_generales = round(costo_directo * gastos_generales, 2)

    # Calcular utilidad (5% del costo directo)
    utilidad = round(costo_directo * 0.05, 2)

    # Calcular subtotal
    subtotal = round(costo_directo + gastos_generales + utilidad, 2)

    # Calcular IGV (18% del subtotal)
    igv = round(subtotal * 0.18, 2)

    # Calcular total
    total = round(subtotal + igv, 2)

    return {
        "costo_directo": round(costo_directo, 2),
        "gastos_generales": gastos_generales,
        "utilidad": utilidad,
        "subtotal": subtotal,
        "igv": igv,
        "total": total,
    }

### Multiplicar el precio unitario actualizado por la carga trabajo


In [131]:
def calcular_costos_unitarios(
    dict_precios_unitarios_actualizados, cargas_trabajo
):
    """
    Multiplica cada carga de trabajo del contratista por su precio unitario
    correspondiente.

    Si alguna clave en cargas_trabajo tiene carga > 0 y no existe
    en dict_precios_unitarios_actualizados, levanta KeyError.
    Claves con carga = 0 se ignoran (no dan error), y claves en precios que noprecios_uninatrios_expediente
    están en cargas se tratan con carga = 0.

    Args:
        dict_precios_unitarios_actualizados (dict): código → precio unitario.
        cargas_trabajo (dict): código → carga de trabajo.

    Returns:
        dict: código → precio total (precio_unitario * carga).

    Raises:
        KeyError: si alguna clave con carga > 0 en cargas_trabajo
                  no existe en dict_precios_unitarios_actualizados.
    """
    # Detectar faltantes solo para cargas > 0
    faltantes = {
        codigo
        for codigo, carga in cargas_trabajo.items()
        if carga != 0 and codigo not in dict_precios_unitarios_actualizados
    }
    if faltantes:
        raise KeyError(f"Faltan precios unitarios para las claves: {faltantes}")

    # Construir resultado: para cada precio, multiplicar por carga (0 si no hay)
    resultado = {}
    for codigo, precio_unitario in dict_precios_unitarios_actualizados.items():
        carga = cargas_trabajo.get(codigo, 0)
        resultado[codigo] = precio_unitario * carga

    return resultado

### Fusionar diccionarios

In [132]:
def fusionar_diccionarios(diccionario_de_diccionarios):
    """
    Fusiona diccionarios separados en un diccionario unificado.
    Solo incluye claves donde ambos valores sean diferentes de cero.
    Usa dinámicamente las claves del diccionario de entrada.
    
    Args:
        diccionario_de_diccionarios (dict): Diccionario con estructura:
            {
                "nombre_campo1": {clave: valor, ...},
                "nombre_campo2": {clave: valor, ...}
            }
    
    Returns:
        dict: Diccionario fusionado con estructura {clave: {nombre_campo1: valor, nombre_campo2: valor}}
    """
    # Obtener las claves (nombres de los campos) del diccionario principal
    nombres_campos = list(diccionario_de_diccionarios.keys())
    
    if len(nombres_campos) != 2:
        raise ValueError("El diccionario debe contener exactamente 2 campos")
    
    campo1_nombre = nombres_campos[0]
    campo2_nombre = nombres_campos[1]
    
    campo1_datos = diccionario_de_diccionarios[campo1_nombre]
    campo2_datos = diccionario_de_diccionarios[campo2_nombre]
    
    fusionado = {}
    
    # Obtener todas las claves únicas de ambos diccionarios
    todas_las_claves = set(campo1_datos.keys()) | set(campo2_datos.keys())
    
    for clave in todas_las_claves:
        valor1 = campo1_datos.get(clave, 0)
        valor2 = campo2_datos.get(clave, 0)
        
        # Solo agregar si ambos valores son diferentes de cero
        if valor1 != 0 and valor2 != 0:
            fusionado[clave] = {
                campo1_nombre: valor1,
                campo2_nombre: valor2
            }
    
    return fusionado

### Formatear progresiva

In [133]:
def formatear_progresiva(distancia, decimales=0):
    """
    Convierte una distancia en metros a notación de progresiva.
    
    Parámetros:
    - distancia: int o float, la distancia en metros.
    - decimales: int, número de decimales a mostrar en la parte de los metros.
    
    Retorna:
    - str: progresiva en formato 'K+XXX' con los decimales indicados.
    """
    if not isinstance(distancia, (int, float)):
        raise ValueError("La distancia debe ser un número (int o float).")
    if not isinstance(decimales, int) or decimales < 0:
        raise ValueError("Los decimales deben ser un entero no negativo.")
    
    km = int(distancia) // 1000
    metros = distancia - (km * 1000)
    
    formato_metros = f"{metros:0.{decimales}f}".zfill(3 + (1 if decimales > 0 else 0) + decimales)
    return f"{km}+{formato_metros}"

### Calcular avance

In [134]:
def calcular_avance(programado, ejecutado):
    """
    Calcula el porcentaje de avance basado en lo programado y lo ejecutado.

    Si lo programado es 0 y lo ejecutado es mayor a 0, devuelve 'ejecución adelantada'.
    Si ambos son 0, devuelve 0.0.
    En cualquier otro caso, devuelve el porcentaje (0-1) como float redondeado a 2 decimales.
    """
    if programado == 0:
        if ejecutado > 0:
            return "Ejec. adelantada"
        else:
            return 0.0
    else:
        porcentaje = (ejecutado / programado) 
        return porcentaje

### Ordenar por centena

In [135]:
def ordenar_por_centena(data: Any) -> List[Dict[str, Any]]:
    """
    Toma un dict (o un JSON en formato str) cuyas claves acaban en número
    y devuelve una lista de dicts {'key':…, 'value':…} ordenada por ese número.
    Compatible con Firestore (to_dict()) y JSON.
    """
    # Si viene como cadena JSON, lo convertimos
    if isinstance(data, str):
        data = json.loads(data)
    
    # Aseguramos que sea dict
    if not isinstance(data, dict):
        raise ValueError("Se esperaba un diccionario o un string JSON que represente un diccionario.")

    pattern = re.compile(r'(\d+)$')

    def obtener_clave_numerica(item):
        clave = str(item[0])  # Convertimos clave a string en caso no lo sea
        match = pattern.search(clave)
        if match:
            return int(match.group(1))
        else:
            return float('inf')  # Opcional: claves sin número al final se van al final

    sorted_items = sorted(data.items(), key=obtener_clave_numerica)

    return [{"key": k, "value": v} for k, v in sorted_items]

### Clave grupo

In [136]:
def clave_grupo(item):
    return int(item['key'][2:]) // 100  # 101→1, 201→2, etc.

## Carga de datos


### Firebase


In [137]:
# 1. Busca el .env en el directorio actual o en cualquiera de los padres
dotenv_path = find_dotenv()
if not dotenv_path:
    raise FileNotFoundError(
        "No se encontró ningún archivo .env en este directorio ni en sus padres."
    )
load_dotenv(dotenv_path)

# 2. Define el root del proyecto como la carpeta que contiene el .env
project_root = Path(dotenv_path).parent

# 3. Obtén la ruta relativa de las credenciales desde la variable de entorno
rel_cred_path = os.getenv("FIRESTORE_CREDENTIALS")
if not rel_cred_path:
    raise RuntimeError("No existe la variable FIRESTORE_CREDENTIALS en el .env")

# 4. Construye la ruta absoluta al JSON
cred_path = Path(rel_cred_path)
if not cred_path.is_absolute():
    cred_path = (project_root / cred_path).resolve()

if not cred_path.exists():
    raise FileNotFoundError(f"No existe el archivo de credenciales en: {cred_path}")

# 5. Carga las credenciales y crea el cliente de Firestore
credentials = service_account.Credentials.from_service_account_file(str(cred_path))
client = firestore.Client(credentials=credentials, project=credentials.project_id)

# 6. Prueba que funcione
print("Colecciones disponibles:", [c.id for c in client.collections()])

Colecciones disponibles: ['rutinarios']


In [138]:
db = firestore.Client(credentials=credentials, project=credentials.project_id)

#### Documento del proyecto firebase

In [139]:
doc_proyecto_firebase = db.collection("rutinarios").document(proyecto).get().to_dict()

pprint.pprint(doc_proyecto_firebase)

{'contrato': {'contratista': {'razon_social': 'BUILDIA S.A.C.', 'ruc': ''},
              'denominacion_tramo_convenio': 'EMP. AR-127 - MISANAYOC (KM '
                                             '6+000)',
              'fecha_inicio': {'anio': 2025, 'dia': 12, 'mes': 4},
              'id_contrato': '002-2025',
              'jefe_mantenimiento': {'apellido': 'Tinta Cáceres',
                                     'dni': 0,
                                     'nombre': 'Genaro',
                                     'titulo': 'Ingeniero'},
              'monto_contrato': 40800.0,
              'numero_cuadrillas': 1,
              'numero_trabajadores': 2,
              'tiempo_ejecucion_dias': 240,
              'tipo_servicio': 'mantenimiento rutinario'},
 'datos_generales': {'distritos': ['Huanca'],
                     'provincia': 'Caylloma',
                     'region': 'Arequipa'},
 'expediente': {'cargas_trabajo': {'MR101': 4.4,
                                   'MR102': 140

#### Contrato


In [140]:
contrato = doc_proyecto_firebase["contrato"]
pprint.pprint(contrato)

{'contratista': {'razon_social': 'BUILDIA S.A.C.', 'ruc': ''},
 'denominacion_tramo_convenio': 'EMP. AR-127 - MISANAYOC (KM 6+000)',
 'fecha_inicio': {'anio': 2025, 'dia': 12, 'mes': 4},
 'id_contrato': '002-2025',
 'jefe_mantenimiento': {'apellido': 'Tinta Cáceres',
                        'dni': 0,
                        'nombre': 'Genaro',
                        'titulo': 'Ingeniero'},
 'monto_contrato': 40800.0,
 'numero_cuadrillas': 1,
 'numero_trabajadores': 2,
 'tiempo_ejecucion_dias': 240,
 'tipo_servicio': 'mantenimiento rutinario'}


In [141]:
monto_contrato = contrato["monto_contrato"]
print(monto_contrato)

40800.0


#### Expediente técnico firebase

In [142]:
expediente_tecnico_firebase = doc_proyecto_firebase["expediente"]

pprint.pprint(expediente_tecnico_firebase)

{'cargas_trabajo': {'MR101': 4.4,
                    'MR102': 1405,
                    'MR103': 8.67,
                    'MR104': 26.67,
                    'MR201': 6000,
                    'MR203': 420,
                    'MR301': 3466.67,
                    'MR401': 13.33,
                    'MR601': 29.04,
                    'MR701': 16},
 'codigo_ruta': 'R0405148',
 'coordenadas': {'fin': {'altitud': 3728,
                         'datum': 'WGS84',
                         'hemisferio': 'S',
                         'progresiva': 6600,
                         'x': 189675.12,
                         'y': 8229710.1,
                         'zona': None,
                         'zona_letra': None},
                 'inicio': {'altitud': 3130,
                            'datum': 'WGS84',
                            'hemisferio': 'S',
                            'progresiva': 0,
                            'x': 191509.1,
                            'y': 8225824.5,
        

In [143]:
gastos_generales_expediente=expediente_tecnico_firebase['gastos_generales']
print(gastos_generales_expediente)


0.15


#### Progresiva de inicio y fin del expediente

In [144]:
progresiva_inicio=expediente_tecnico_firebase["coordenadas"]["inicio"]["progresiva"]
progresiva_fin=expediente_tecnico_firebase["coordenadas"]["fin"]["progresiva"]

print(progresiva_inicio)
print(progresiva_fin)

0
6600


#### Valorización programada mensual


In [145]:
desembolsos =  db.collection("rutinarios").document(proyecto).collection('presupuestos').document('desembolsos').get().to_dict()
cronogramas_desembolsos = desembolsos['cronograma_desembolsos']
pprint.pprint(cronogramas_desembolsos)

{'10': {'gastos_operativos': 527.0,
        'igv': 948.6,
        'mantenimiento_con_go': 5797.0,
        'mantenimiento_con_igv': 5270.0,
        'mantenimiento_sin_igv': 4321.4},
 '11': {'gastos_operativos': 510.0,
        'igv': 918.0,
        'mantenimiento_con_go': 5610.0,
        'mantenimiento_con_igv': 5100.0,
        'mantenimiento_sin_igv': 4182.0},
 '12': {'gastos_operativos': 119.0,
        'igv': 214.2,
        'mantenimiento_con_go': 1309.0,
        'mantenimiento_con_igv': 1190.0,
        'mantenimiento_sin_igv': 975.8},
 '4': {'gastos_operativos': 323.0,
       'igv': 581.4,
       'mantenimiento_con_go': 3553.0,
       'mantenimiento_con_igv': 3230.0,
       'mantenimiento_sin_igv': 2648.6},
 '5': {'gastos_operativos': 527.0,
       'igv': 948.6,
       'mantenimiento_con_go': 5797.0,
       'mantenimiento_con_igv': 5270.0,
       'mantenimiento_sin_igv': 4321.4},
 '6': {'gastos_operativos': 510.0,
       'igv': 918.0,
       'mantenimiento_con_go': 5610.0,
       'man

In [146]:
desembolso_current_month = cronogramas_desembolsos[str(mes)]
print(desembolso_current_month["mantenimiento_con_igv"])

5270.0


#### Cargas de trabajo mensual presentadas por el contratista


In [147]:
valorizaciones = db.collection("rutinarios").document(proyecto).collection('valorizaciones').document(str(mes)).get().to_dict()
cargas_trabajo_contratista = valorizaciones['cargas_trabajo_contratista_inicial']
print(cargas_trabajo_contratista)

{'MR301': 385.19, 'MR203': 63.64, 'MR201': 769.23, 'MR101': 0.57, 'MR102': 180.46, 'MR104': 3.33, 'MR701': 2, 'MR601': 4.15}


#### Cargas de trabajo del expediente técnico

In [148]:
cargas_trabajo_expediente_tecnico=doc_proyecto_firebase['expediente']['cargas_trabajo']

pprint.pprint(cargas_trabajo_expediente_tecnico)

{'MR101': 4.4,
 'MR102': 1405,
 'MR103': 8.67,
 'MR104': 26.67,
 'MR201': 6000,
 'MR203': 420,
 'MR301': 3466.67,
 'MR401': 13.33,
 'MR601': 29.04,
 'MR701': 16}


#### Precios unitarios del expediente técnico

In [149]:
precios_unitarios_expediente_tecnico = doc_proyecto_firebase['expediente']['precios_unitarios']

pprint.pprint(precios_unitarios_expediente_tecnico)

{'MR101': 265.3,
 'MR102': 10.27,
 'MR103': 21.0,
 'MR104': 17.5,
 'MR201': 0.44,
 'MR202': 78.5,
 'MR203': 5.25,
 'MR204': 0.44,
 'MR205': 105,
 'MR206': 3.6,
 'MR301': 0.14,
 'MR401': 11.42,
 'MR501': 0.53,
 'MR601': 2.1,
 'MR701': 43.75,
 'MR702': 210}


### Pickle

#### Cargas de trabajo programadas

In [150]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_cargas_trabajo.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    cargas_trabajo_programadas_anualmente = pickle.load(f)

cargas_trabajo_programadas_anualmente.tail(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
codigo,,,,,,,,,,
MR103,0.000000,0.000000,2.890000,0.000000,0.000000,2.890000,0.000000,2.890000,0.000000,8.67
MR104,3.333750,3.333750,3.333750,3.333750,3.333750,3.333750,3.333750,3.333750,0.000000,26.67
MR201,461.538462,769.230769,769.230769,769.230769,769.230769,769.230769,769.230769,769.230769,153.846154,6000.00
MR202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR203,38.181818,63.636364,50.909091,50.909091,50.909091,50.909091,50.909091,50.909091,12.727273,420.00
MR204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR206,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR301,385.185556,385.185556,385.185556,385.185556,385.185556,385.185556,385.185556,385.185556,385.185556,3466.67


In [151]:
#eliminando el total
cargas_trabajo_programadas_anualmente = cargas_trabajo_programadas_anualmente.iloc[:-1]

In [152]:
cargas_trabajo_programadas_current_month = cargas_trabajo_programadas_anualmente[f"2025-{mes:02d}"].to_dict()
pprint.pprint(cargas_trabajo_programadas_current_month)

{'MR101': 0.5739130434782609,
 'MR102': 180.45871559633028,
 'MR103': 0.0,
 'MR104': 3.33375,
 'MR201': 769.2307692307692,
 'MR202': 0.0,
 'MR203': 63.63636363636364,
 'MR204': 0.0,
 'MR205': 0.0,
 'MR206': 0.0,
 'MR301': 385.18555555555554,
 'MR401': 3.3325,
 'MR501': 0.0,
 'MR601': 0.0,
 'MR701': 2.0,
 'MR702': 0.0}


In [153]:
cargas_trabajo_programadas_anualmente.index.to_list()

['MR101',
 'MR102',
 'MR103',
 'MR104',
 'MR201',
 'MR202',
 'MR203',
 'MR204',
 'MR205',
 'MR206',
 'MR301',
 'MR401',
 'MR501',
 'MR601',
 'MR701',
 'MR702']

#### Cronograma anual

In [154]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_cronograma_anual.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    df_cronograma_anual = pickle.load(f)

df_cronograma_anual.head(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
MR101,1,3,3,3,3,3,3,3,1,23
MR102,9,14,14,14,14,13,14,14,3,109
MR103,0,0,1,0,0,1,0,1,0,3
MR104,1,1,1,1,1,1,1,1,0,8
MR201,3,5,5,5,5,5,5,5,1,39
MR202,0,0,0,0,0,0,0,0,0,0
MR203,3,5,4,4,4,4,4,4,1,33
MR204,0,0,0,0,0,0,0,0,0,0
MR205,0,0,0,0,0,0,0,0,0,0
MR206,0,0,0,0,0,0,0,0,0,0


### JSON

#### Actividades

In [155]:
ruta_actividades= os.path.join("data", "general_data", "actividades.json")
with open(ruta_actividades, 'r', encoding='utf-8') as archivo:
    actividades = json.load(archivo)
# Ahora 'datos' es un diccionario de Python
print(actividades)

[{'key': 'MR100', 'value': {'label': 'Conservación de calzada', 'value': [{'key': 'MR101', 'value': {'label': 'Limpieza de calzada', 'carga_trabajo': 0, 'unidad': 'Km'}}, {'key': 'MR102', 'value': {'label': 'Bacheo', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR103', 'value': {'label': 'Desquinche', 'carga_trabajo': 0, 'unidad': 'm3'}}, {'key': 'MR104', 'value': {'label': 'Remoción de derrumbes', 'carga_trabajo': 0, 'unidad': 'm3'}}]}}, {'key': 'MR200', 'value': {'label': 'Limpieza de obras de arte', 'value': [{'key': 'MR201', 'value': {'label': 'Limpieza de cunetas', 'carga_trabajo': 1440, 'unidad': 'm'}}, {'key': 'MR202', 'value': {'label': 'Limpieza de alcantarillas', 'carga_trabajo': 0, 'unidad': 'unidad'}}, {'key': 'MR203', 'value': {'label': 'Limpieza de badén', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR204', 'value': {'label': 'Limpieza de zanjas de coronación', 'carga_trabajo': 0, 'unidad': 'm'}}, {'key': 'MR205', 'value': {'label': 'Limpieza de pontones', 'carga_tra

## Cálculos


### Cargas de trabajo y pu de expediente tecnico

In [156]:
datos_cargas_trabajo_expediente_tecnico=fusionar_diccionarios(
    {
        'precio_unitario':precios_unitarios_expediente_tecnico,
        'carga_trabajo':cargas_trabajo_expediente_tecnico
    }
)

pprint.pprint(datos_cargas_trabajo_expediente_tecnico)

{'MR101': {'carga_trabajo': 4.4, 'precio_unitario': 265.3},
 'MR102': {'carga_trabajo': 1405, 'precio_unitario': 10.27},
 'MR103': {'carga_trabajo': 8.67, 'precio_unitario': 21.0},
 'MR104': {'carga_trabajo': 26.67, 'precio_unitario': 17.5},
 'MR201': {'carga_trabajo': 6000, 'precio_unitario': 0.44},
 'MR203': {'carga_trabajo': 420, 'precio_unitario': 5.25},
 'MR301': {'carga_trabajo': 3466.67, 'precio_unitario': 0.14},
 'MR401': {'carga_trabajo': 13.33, 'precio_unitario': 11.42},
 'MR601': {'carga_trabajo': 29.04, 'precio_unitario': 2.1},
 'MR701': {'carga_trabajo': 16, 'precio_unitario': 43.75}}


In [157]:
# Crear DataFrame usando pd.DataFrame.from_dict() con orient='index'
df_cargas_trabajo_expediente = pd.DataFrame.from_dict(
    datos_cargas_trabajo_expediente_tecnico, orient="index"
)

# Resetear el índice para convertir las claves MR en una columna
df_cargas_trabajo_expediente = df_cargas_trabajo_expediente.reset_index().rename(
    columns={"index": "codigo_MR"}
)

In [158]:
# Agregar columna parcial
df_cargas_trabajo_expediente["parcial"] = (
    df_cargas_trabajo_expediente["precio_unitario"]
    * df_cargas_trabajo_expediente["carga_trabajo"]
)

In [159]:
df_cargas_trabajo_expediente.head(15)

,codigo_MR,precio_unitario,carga_trabajo,parcial
0,MR104,17.50,26.67,466.7250
1,MR103,21.00,8.67,182.0700
2,MR301,0.14,3466.67,485.3338
3,MR401,11.42,13.33,152.2286
4,MR203,5.25,420.00,2205.0000
5,MR701,43.75,16.00,700.0000
6,MR101,265.30,4.40,1167.3200
7,MR102,10.27,1405.00,14429.3500
8,MR201,0.44,6000.00,2640.0000
9,MR601,2.10,29.04,60.9840


In [160]:
# Calcular el total
costo_directo = df_cargas_trabajo_expediente["parcial"].sum()
print("costo_directo", costo_directo)

costo_directo 22489.0114


In [161]:
costo_total_expediente = calculadora_costo_total(costo_directo,gastos_generales_expediente)
print(costo_total_expediente["total"])

31844.44


In [162]:
# Agregar columna parcial
df_cargas_trabajo_expediente["precio_unitario_actualizado"] = (
    df_cargas_trabajo_expediente["precio_unitario"]
    * (monto_contrato / costo_total_expediente["total"])
)

df_cargas_trabajo_expediente["parcial_actualizado"] = (
    df_cargas_trabajo_expediente["precio_unitario_actualizado"]
    * df_cargas_trabajo_expediente["carga_trabajo"]
)

# Calcular el total
costo_directo_actualizado = df_cargas_trabajo_expediente["parcial_actualizado"].sum()

print(f"El costo directo es: {costo_directo_actualizado}")

El costo directo es: 28813.559450880595


In [163]:
df_cargas_trabajo_expediente.tail(10)

,codigo_MR,precio_unitario,carga_trabajo,parcial,precio_unitario_actualizado,parcial_actualizado
0,MR104,17.50,26.67,466.7250,22.421496,597.981312
1,MR103,21.00,8.67,182.0700,26.905796,233.273250
2,MR301,0.14,3466.67,485.3338,0.179372,621.823434
3,MR401,11.42,13.33,152.2286,14.631628,195.039601
4,MR203,5.25,420.00,2205.0000,6.726449,2825.108559
5,MR701,43.75,16.00,700.0000,56.053741,896.859860
6,MR101,265.30,4.40,1167.3200,339.909887,1495.603503
7,MR102,10.27,1405.00,14429.3500,13.158215,18487.292601
8,MR201,0.44,6000.00,2640.0000,0.563740,3382.442901
9,MR601,2.10,29.04,60.9840,2.690580,78.134431


In [164]:
pago_costo_total_contratista = calculadora_costo_total(costo_directo_actualizado,gastos_generales_expediente)
print(pago_costo_total_contratista["total"])

40800.0


In [165]:
dict_precios_unitarios_actualizados = dict(
    zip(
        df_cargas_trabajo_expediente["codigo_MR"],
        df_cargas_trabajo_expediente["precio_unitario_actualizado"],
    )
)

pprint.pprint(dict_precios_unitarios_actualizados)

{'MR101': 339.9098869378768,
 'MR102': 13.158215374489236,
 'MR103': 26.905795799831935,
 'MR104': 22.421496499859945,
 'MR201': 0.5637404834250501,
 'MR203': 6.726448949957984,
 'MR301': 0.17937197199887958,
 'MR401': 14.63162800162289,
 'MR601': 2.6905795799831935,
 'MR701': 56.05374124964987}


### Cálculo de pago de acuerdo a cargas de trabajo


In [166]:
pago_costo_directo_parciales_contratista = calcular_costos_unitarios(
    dict_precios_unitarios_actualizados, cargas_trabajo_contratista
)
pprint.pprint(pago_costo_directo_parciales_contratista)

{'MR101': 193.74863555458978,
 'MR102': 2374.5315464803275,
 'MR103': 0.0,
 'MR104': 74.66358334453362,
 'MR201': 433.64609206505133,
 'MR203': 428.0712111753261,
 'MR301': 69.09228989424842,
 'MR401': 0.0,
 'MR601': 11.165905256930253,
 'MR701': 112.10748249929973}


##### Visualizacion en dataframe


In [167]:
df_pago_costo_directo_parciales_contratista = pd.DataFrame.from_dict(
    pago_costo_directo_parciales_contratista, orient="index"
)

df_pago_costo_directo_parciales_contratista = (
    df_pago_costo_directo_parciales_contratista.reset_index().rename(
        columns={"index": "codigo_MR"}
    )
)

df_pago_costo_directo_parciales_contratista = (
    df_pago_costo_directo_parciales_contratista.rename(columns={0: "monto_pago"})
)

df_pago_costo_directo_parciales_contratista

,codigo_MR,monto_pago
0,MR104,74.663583
1,MR103,0.000000
2,MR301,69.092290
3,MR401,0.000000
4,MR203,428.071211
5,MR701,112.107482
6,MR101,193.748636
7,MR102,2374.531546
8,MR201,433.646092
9,MR601,11.165905


In [168]:
# Ordenar el DataFrame por la columna 'monto_pago'
# Por defecto, el orden es ascendente (de menor a mayor)
df_ordenado = df_pago_costo_directo_parciales_contratista.sort_values(
    by="monto_pago", ascending=False
)

print("\nDataFrame Ordenado por 'monto_pago' (ascendente):")
print(df_ordenado)


DataFrame Ordenado por 'monto_pago' (ascendente):
  codigo_MR   monto_pago
7     MR102  2374.531546
8     MR201   433.646092
4     MR203   428.071211
6     MR101   193.748636
5     MR701   112.107482
0     MR104    74.663583
2     MR301    69.092290
9     MR601    11.165905
1     MR103     0.000000
3     MR401     0.000000


In [169]:
# Lambda que suma todos los valores de un diccionario
sumar_valores = lambda d: sum(d.values())

In [170]:
pago_costo_directo_contratista = sumar_valores(pago_costo_directo_parciales_contratista)
print(pago_costo_directo_contratista)

3697.026746270307


In [171]:
pago_costo_total_contratista = calculadora_costo_total(pago_costo_directo_contratista,gastos_generales_expediente)
print(pago_costo_total_contratista["total"])

5234.99


##### Validación


In [172]:
diferencia_costos = abs(
    pago_costo_total_contratista["total"]
    - desembolso_current_month["mantenimiento_con_igv"]
)

pprint.pprint(
    {
        "diferencia_costos": diferencia_costos,
        "ejecutado": pago_costo_total_contratista["total"],
        "programado": desembolso_current_month["mantenimiento_con_igv"],
    }
)

if diferencia_costos > 1:
    raise ValueError(
        "La valorizacion de las actividades presentadas por el contratista no es coherente con lo programado"
    )

{'diferencia_costos': 35.01000000000022,
 'ejecutado': 5234.99,
 'programado': 5270.0}


ValueError: La valorizacion de las actividades presentadas por el contratista no es coherente con lo programado

### Ordenando cargas de trabajo

In [ ]:
cargas_trabajo_ordenadas=ordenar_por_centena(cargas_trabajo_programadas_current_month)
pprint.pprint(cargas_trabajo_ordenadas)

[{'key': 'MR101', 'value': 0.99375},
 {'key': 'MR102', 'value': 328.0038461538461},
 {'key': 'MR103', 'value': 5.0075},
 {'key': 'MR104', 'value': 10.17142857142857},
 {'key': 'MR201', 'value': 0.0},
 {'key': 'MR202', 'value': 0.0},
 {'key': 'MR203', 'value': 44.448},
 {'key': 'MR204', 'value': 0.0},
 {'key': 'MR205', 'value': 0.0},
 {'key': 'MR206', 'value': 33.91},
 {'key': 'MR301', 'value': 3903.83},
 {'key': 'MR401', 'value': 0.0},
 {'key': 'MR501', 'value': 0.0},
 {'key': 'MR601', 'value': 0.0},
 {'key': 'MR701', 'value': 3.4320000000000004},
 {'key': 'MR702', 'value': 0.0}]
